# Programming Assignment 2

**Name:** Venkateswar Reddy Melachervu<br />
**Roll No:23156022** 


## E1: A Deterministic Career Path

Consider a simple Markov Decision Process below with four states and two actions available at each state. In this simplistic setting actions have deterministic effects, i.e., taking an action in a state always leads to one next state with transition probability equal to one. There are two actions out of each state for the agent to choose from: D for development and R for research. The _ultimately-care-only-about-money_ reward scheme is given along with the states.

<img src='assets/mdp-d.png' width="700" align="left"></img>

In [2]:
# import required libraries
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random
import time

### E1.1 Environment Implementation

In [3]:
'''
Represents a Career Path problem Gym Environment which provides a Fully observable
MDP
'''
class CareerPathEnv(gym.Env):
    '''
    CareerPathEnv represents the Gym Environment for the Career Path problem environment
    States : [0:'Unemployed', 1:'Industry', 2:'Grad School', 3:'Academia']
    Actions : [0:'Research', 1:'Development']
    '''
    metadata = {'render.modes': ['human']}

    state_names = {0:'Unemployed', 1:'Industry', 2:'Grad School', 3:'Academia'}
    action_names = { 0:'Research', 1:'Development'}

    def __init__(self,initial_state=0,no_states=4,no_actions=2):
        '''
        Constructor for the CareerPath class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 4
            no_actions : The no. of possible actions which is 2
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.prob_dynamics = {
            # s: {
            #   a: [(p(s,s'|a), s', r', terminal/not)]    
            # }

            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(1.0, 0, -10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        # print("Current state: {}".format(self.state))
        print(f"Current state: {self.state} - {self.state_names.get(self.state)}")

    def render_action(self, action):
        '''
        Renders the action specified
        '''
        print(f"Current action: {action} - {self.state_names.get(action)}")

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''

        # s: {
        #   a: [(p(s,s'|a), s', r', terminal/not)]    
        # }
        self.prob_dynamics = {
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(1.0, 0, -10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        dynamics_tuple = self.prob_dynamics[self.state][action][0]
        self.state = dynamics_tuple[1]

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

### E1.2 Policies

After implementing the environment let us see how to make decisions in the environment. Let $\pi_1(s) = R$ and $\pi_2(s) = D$ for any state be two policies. Let us see how these policies look like.

In [4]:
policy_R = np.concatenate((np.ones([4, 1]), np.zeros([4, 1])), axis=1)
policy_D = np.concatenate((np.zeros([4, 1]), np.ones([4, 1])), axis=1)
policy_random = np.array((np.random.permutation(2), np.random.permutation(2), np.random.permutation(2), np.random.permutation(2)))
print("Research policy: \n",policy_R)
print("Development policy: \n", policy_D)
print("Random policy: \n",policy_random)
policy_uncertain = np.concatenate((0.5*np.ones([4, 1]), 0.5*np.ones([4, 1])), axis=1)
print("Uncertain policy: \n",policy_uncertain)

Research policy: 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Development policy: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Random policy: 
 [[0 1]
 [0 1]
 [1 0]
 [0 1]]
Uncertain policy: 
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


### E1.3 Testing

By usine one of the above policies, lets see how we navigate the environment. We want to see how we take action based on a given policy, what state we transition to and obtain the rewards from the transition.

In [5]:
# env = CareerPathEnv()
env = CareerPathEnv(initial_state=1, no_states=4, no_actions=2)
is_Terminal = False
print('Resetting the environment...')
start_state = env.reset()
print('Environment is reset.')
env.render()

steps = 0
total_reward = 0

# you may change policy here
# policy = policy_uncertain
policy = policy_D
selected_policy = 'Development Policy'
# policy = policy_D
# policy = policy_random
print('Selected policy is:', selected_policy)
print("State\t", "Action\t" , "New State\t", "Reward\t" , "is_Terminal")
steps = 0
max_steps = 5

prev_state = start_state

while steps < 10:
    steps += 1

    action = np.random.choice(2,1,p=policy[prev_state])[0]  #0 -> Research, 1 -> Development
    state, reward, is_Terminal = env.step(action)
    
    total_reward += reward

    print(" ",prev_state, "\t  ", action, "\t  ", state, "\t\t", reward, "\t  ", is_Terminal)
    prev_state = state
    
print("Total Number of steps:", steps)
print("Final Reward:", total_reward)

Resetting the environment...
Environment is reset.
Current state: 1 - Industry
Selected policy is: Development Policy
State	 Action	 New State	 Reward	 is_Terminal
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
  1 	   1 	   1 		 100.0 	   False
Total Number of steps: 10
Final Reward: 1000.0


### Iterative Policy Evaluation
Iterative Policy Evaluation is commonly used to calculate the state value function $V_\pi(s)$ for a given policy $\pi$. Here we implement a function to compute the state value function $V_\pi(s)$ for a given policy

<img src='assets/policy_eval.png' width="500" align="left"></img>

In [6]:
# Policy Evaluation
def EvaluatePolicy(env, policy, gamma=0.9, theta=1e-8, draw=False):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            Vs = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P()[s][a]:
                    Vs += action_prob * prob * (reward + gamma * V[next_state])
            delta = max(delta, np.abs(V[s]-Vs))
            V[s] = Vs
        if delta < theta:
            break
    return V

### Policy improvement

$\pi'(s) = \arg \max_a \sum_{s',r} p(s',r|s,a)\left[ r + \gamma v_\pi(s') \right ]$


In [7]:
##Policy Improvement Function
def ImprovePolicy(env, v, gamma):
    num_states = env.nS
    num_actions = env.nA
    prob_dynamics = env.P()

    q = np.zeros((num_states, num_actions))
    
    for state in prob_dynamics:
            for action in prob_dynamics[state]:
                #print(state, action)
                for prob, new_state, reward, is_terminal in prob_dynamics[state][action]:
                    #print(prob, new_state, reward, is_terminal)
                    q[state][action] += prob*(reward + gamma*v[new_state])
                        
    new_pi = np.zeros((num_states, num_actions))
    
    for state in range(num_states):
        opt_action = np.argmax(q[state])
        new_pi[state][opt_action] = 1.0
    
    return new_pi       

### Policy Iteration

<img src='assets/policy_iteration.png' width="500" align="left"></img>

In [8]:
# Policy iteration function
def PolicyIteration(env, pi, gamma, tol = 1e-10):
    num_states = env.nS
    num_actions = env.nA
    iterations = 0
    
    while True:
        # print(pi)
        iterations += 1
        pi_old = pi
        v = EvaluatePolicy(env, pi_old, gamma, tol)
        pi = ImprovePolicy(env, v, gamma)
        
        is_equal = True
        for s in range(num_states):
            if np.argmax(pi_old[s]) == np.argmax(pi[s]): 
                continue
            is_equal = False
        if is_equal == True:
            break
    return pi, v, iterations

 

### Testing Policy Iteration 

In [9]:
import time

# Test/iterate the chosen policy for improvement 
gamma = 0.9
env = CareerPathEnv()
chosen_policy_to_improve = policy_random
print("Initializing the environment for improving the policy using Policy Iteration...")
print("Initial Policy Details:")
print("State and Actions Values: \n", chosen_policy_to_improve)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(chosen_policy_to_improve[s])} ({env.action_names[np.argmax(chosen_policy_to_improve[s])]})")
print()
print("Improving the policy using Policy Iteration...")
print()
pi, pi_v, pi_iters = PolicyIteration(env, chosen_policy_to_improve, gamma)
print("Improved/Iterated Policy Details:")
print("State and Actions Values: \n", pi)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(pi[s])} ({env.action_names[np.argmax(pi[s])]})")
print("State Values: ", pi_v)
print("Number of iterations: ",pi_iters)

# average number of iterations required
pi_avg_iters = 0
pi_min_iters = 1000
pi_max_iters = 0
pi_start_time = time.time()
r = 100
for _ in range(r):
    policy_random = np.array((np.random.permutation(2), np.random.permutation(2), np.random.permutation(2), np.random.permutation(2)))
    _, _, iters = PolicyIteration(env,policy_random, gamma)
    pi_avg_iters += iters
    pi_min_iters = min(pi_min_iters, iters)
    pi_max_iters = max(pi_max_iters, iters)
pi_end_time = time.time()
pi_avg_iters /= 100
pi_avg_iter_time = (pi_end_time - pi_start_time) / pi_avg_iters

print("Iterations:")
print("Min\t", "Max\t" , "Average\t", "Avg Iteration Time(S)")
print(pi_min_iters,"\t", pi_max_iters,"\t", pi_avg_iters, "\t\t", pi_avg_iter_time)

Initializing the environment for improving the policy using Policy Iteration...
Initial Policy Details:
State and Actions Values: 
 [[0 1]
 [0 1]
 [1 0]
 [0 1]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 1 (Development)
State 1 (Industry): Best Action -> 1 (Development)
State 2 (Grad School): Best Action -> 0 (Research)
State 3 (Academia): Best Action -> 1 (Development)

Improving the policy using Policy Iteration...

Improved/Iterated Policy Details:
State and Actions Values: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 1 (Development)
State 1 (Industry): Best Action -> 1 (Development)
State 2 (Grad School): Best Action -> 1 (Development)
State 3 (Academia): Best Action -> 1 (Development)
State Values:  [1000. 1000. 1000. 1000.]
Number of iterations:  2
Iterations:
Min	 Max	 Average	 Avg Iteration Time(S)
1 	 2 	 1.97 		 1.0573211660239903


***

### A1. Find an optimal policy to navigate the given environment using Value Iteration (VI)

<img src='assets/value_iteration.png' width="500" align="left"></img>

In [10]:
# Value Iteration function
def value_iteration(env, gamma=0.9, theta=1e-8):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = V[s]
            V[s] = max(sum(p * (r + gamma * V[s_]) for p, s_, r, _ in env.prob_dynamics[s][a]) for a in range(env.nA))
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        q_values = np.zeros(env.nA)
        for a in range(env.nA):
            q_values[a] = sum(p * (r + gamma * V[s_]) for p, s_, r, _ in env.prob_dynamics[s][a])
        best_action = np.argmax(q_values)
        policy[s, best_action] = 1.0
    return policy, V

### Testing Value Iterations

In [11]:

import time

# Finding optimal policy using value iteration - turning Bellman optimality into update rule.

# Initialize the environment
print("Initializing the environment for improving the policy using Value Iteration...")
gamma = 0.9
env = CareerPathEnv()

# Perform value iteration
print()
print("Finding optimal policy using Value Iteration - Bellman optimality with update rule...")

vi_time_total = 0
for _ in range(r): 
    vi_start_time = time.time()
    vi, vi_state_values = value_iteration(env, gamma)
    vi_end_time = time.time()
    vi_time_elapsed = vi_end_time - vi_start_time
    vi_time_total += vi_time_elapsed
print()
vi_avg_time = vi_time_total/r

# Display the results
print("Value Iterated Policy Details:")
print("State and Actions Values: \n", vi)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(vi[s])} ({env.action_names[np.argmax(vi[s])]})")
print("State Values: ", vi_state_values)
print("Number of Iterations: ", r)
print("Avg Time Per Iteration (S):", vi_avg_time)

Initializing the environment for improving the policy using Value Iteration...

Finding optimal policy using Value Iteration - Bellman optimality with update rule...

Value Iterated Policy Details:
State and Actions Values: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 1 (Development)
State 1 (Industry): Best Action -> 1 (Development)
State 2 (Grad School): Best Action -> 1 (Development)
State 3 (Academia): Best Action -> 1 (Development)
State Values:  [999.99999991 999.99999991 999.99999992 999.99999992]
Number of Iterations:  100
Avg Time Per Iteration (S): 0.0039990472793579104


In [20]:
# Let's compare policy iteration and value iteration results and convergence
print("Comparison of PI and VI in Deterministic Transition MDP")
print("Policy Iteration - Iterated Policy:")
pi, pi_v, pi_iters = PolicyIteration(env, chosen_policy_to_improve, gamma)
print(pi)
print("Policy Iteration - State Value Function:")
print(pi_v)
print("Number of Iterations:", pi_iters)
print()

print("Value Iteration - State and Action Values:")
print(vi)
print("Value Iteration State Value Function:")
print(vi_state_values)

# Render optimized policy in human readable form
print()
print("Comparison of States and Action:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}):")
    print(f"  Value Iteration Best Action -> {np.argmax(vi[s])} ({env.action_names[np.argmax(vi[s])]})")
    print(f"  Policy Iteration Best Action -> {np.argmax(pi[s])} ({env.action_names[np.argmax(pi[s])]})")

print("Averare time taken per iteration by Policy Iteration (S):", pi_avg_iter_time)
print("Averare time taken per iteration by Value Iteration (S):", vi_avg_time)

Comparison of PI and VI in Deterministic Transition MDP
Policy Iteration - Iterated Policy:
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Policy Iteration - State Value Function:
[1000.         1000.         1000.          990.10989011]
Number of Iterations: 2

Value Iteration - State and Action Values:
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Value Iteration State Value Function:
[999.99999991 999.99999991 999.99999992 999.99999992]

Comparison of States and Action:
State 0 (Unemployed):
  Value Iteration Best Action -> 1 (Development)
  Policy Iteration Best Action -> 1 (Development)
State 1 (Industry):
  Value Iteration Best Action -> 1 (Development)
  Policy Iteration Best Action -> 1 (Development)
State 2 (Grad School):
  Value Iteration Best Action -> 1 (Development)
  Policy Iteration Best Action -> 1 (Development)
State 3 (Academia):
  Value Iteration Best Action -> 1 (Development)
  Policy Iteration Best Action -> 1 (Development)
Averare time taken per iteration by Policy Iteration (S): 1.

### A1.2 Compare PI and VI in terms of convergence (average number of iteration, time required for each iteration). Is the policy obtained by both same?
- As can be seen above, the average time taken per iteration is **far less - 0.057 Seconds - for Value Iteration** vis-a-vis Policy Iteration  1.127 Seconds. 

- **The policy is deterministic in this case and hence Value Iteration does not need multiple iterations and will have convergence in one iteration.**

- The optimal policy obtained for **both the cases is same** , as can be seen in the above convergence data.

***

## Part B : A Stochastic Career Path

Now consider a more realistic Markov Decision Process below with four states and two actions available at each state. In this setting Actions have nondeterministic effects, i.e., taking an action in a state always leads to one next state, but which state is the one next state is determined by transition probabilities. These transition probabilites are shown in the figure attached to the transition arrows from states and actions to states. There are two actions out of each state for the agent to choose from: D for development and R for research. The same _ultimately-care-only-about-money_ reward scheme is given along with the states.

<img src='assets/mdp-nd.png' width="700" align="left"></img>

In [13]:
'''
Represents a Career Path problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticCareerPathEnv(gym.Env):
    '''
    StocasticCareerPathEnv represents the Gym Environment for the Career Path problem environment
    States : [0:'Unemployed',1:'Industry',2:'Grad School',3:'Academia']
    Actions : [0:'Research', 1:'Development']
    '''
    metadata = {'render.modes': ['human']}

    state_names = {0:'Unemployed', 1:'Industry', 2:'Grad School', 3:'Academia'}
    action_names = { 0:'Research', 1:'Development'}

    def __init__(self,initial_state=3,no_states=4,no_actions=2):
        '''
        Constructor for the CareerPath class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 4
            no_actions : The no. of possible actions which is 2
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.prob_dynamics = {
            # s: {
            #   a: [(p(s,s'|a), s', r', terminal/not), (p(s,s''|a), s'', r'', terminal/not)]    
            # }
            
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(0.9, 0, -10.0, False),(0.1, 1, 100, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(0.9, 3, 10.0, False),(0.1, 2, 0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 1, 100, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 3, 10, False)],
            },
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def render_action(self, action):
        '''
        Renders the action specified
        '''
        print(f"Current action: {action} - {self.state_names.get(action)}")

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(0.9, 0, -10.0, False),(0.1, 1, 100, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(0.9, 3, 10.0, False),(0.1, 2, 0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 1, 100, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 3, 10, False)],
            },
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        if self.state == 0 or (self.state == 1 and action == 1) or (self.state == 3 and action == 0):
            index = 0    
        else:
            index = np.random.choice(2,1,p=[0.9,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

### Navigating in Stochastic Career Path 

In [14]:
# Stochastic Career Path MDP and navigation 
class StochasticCareerPathEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    state_names = {0: 'Unemployed', 1: 'Industry', 2: 'Grad School', 3: 'Academia'}
    action_names = {0: 'Research', 1: 'Development'}

    def __init__(self, initial_state=0, no_states=4, no_actions=2):
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.prob_dynamics = {
            0: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 1, 100.0, False)]},
            1: {0: [(0.9, 0, -10.0, False), (0.1, 1, 100, False)], 1: [(1.0, 1, 100.0, False)]},
            2: {0: [(0.9, 3, 10.0, False), (0.1, 2, 0, False)], 1: [(0.9, 1, 100.0, False), (0.1, 1, 100, False)]},
            3: {0: [(1.0, 3, 10.0, False)], 1: [(0.9, 1, 100.0, False), (0.1, 3, 10, False)]},
        }
        # self.print_prob_dynamics()
        self.reset()

    def reset(self):
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        return self.state

    def render(self, mode='human'):
        print("Current state: {}".format(self.state))

    def render_action(self, action):
        print(f"Current action: {action} - {self.state_names.get(action)}")

    def sample_action(self):
        return np.random.randint(0, self.nA)        

    def P(self):
        return self.prob_dynamics

    def step(self, action):
        if action >= self.nA:
            action = self.nA - 1

        if self.state == 0 or (self.state == 1 and action == 1) or (self.state == 3 and action == 0):
            index = 0
        else:
            index = np.random.choice(2, 1, p=[0.9, 0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        return self.state, dynamics_tuple[2], dynamics_tuple[3]
    
    def print_prob_dynamics(self):
        for state, actions in self.prob_dynamics.items():
            print(f"State {state}:")
            for action, outcomes in actions.items():
                print(f"  Action {action}:")
                for prob, next_state, reward, terminal in outcomes:
                    terminal_str = "Terminal" if terminal else "Non-terminal"
                    print(f"    -> (Prob: {prob}, Next State: {next_state}, Reward: {reward}, {terminal_str})")

def evaluate_policy(env, policy, gamma=0.9, theta=1e-8):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.prob_dynamics[s][a]:
                    v += action_prob * prob * (reward + gamma * V[next_state])
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return V

def improve_policy(env, V, gamma=0.9):
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        q_values = np.zeros(env.nA)
        for a in range(env.nA):
            q_values[a] = sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, done in env.prob_dynamics[s][a])
        best_action = np.argmax(q_values)
        policy[s, best_action] = 1.0
    return policy

def policy_iteration(env, policy, gamma=0.9, theta=1e-8):
    stable = False
    iterations = 0
    while not stable:
        iterations += 1
        V = evaluate_policy(env, policy, gamma, theta)
        new_policy = improve_policy(env, V, gamma)
        if np.all(policy == new_policy):
            stable = True
        policy = new_policy
    return policy, V, iterations

def value_iteration(env, gamma=0.9, theta=1e-8):
    V = np.zeros(env.nS)
    iterations = 0
    while True:
        iterations += 1
        delta = 0
        for s in range(env.nS):
            v = V[s]
            V[s] = max(sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, done in env.prob_dynamics[s][a]) for a in range(env.nA))
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        q_values = np.zeros(env.nA)
        for a in range(env.nA):
            q_values[a] = sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, done in env.prob_dynamics[s][a])
        best_action = np.argmax(q_values)
        policy[s, best_action] = 1.0
    return policy, V, iterations

def measure_convergence(env, policy, method, gamma=0.9):
    start_time = time.time()
    if method == "VI":
        policy, V, iterations = value_iteration(env, gamma)
    elif method == "PI":
        policy, V, iterations = policy_iteration(env, policy, gamma)
    else:
        raise ValueError("Invalid method specified.")
    end_time = time.time()
    return policy, V, iterations, (end_time - start_time)

In [15]:
# Let's test SCP
policy_R = np.concatenate((np.ones([4, 1]), np.zeros([4, 1])), axis=1)
policy_D = np.concatenate((np.zeros([4, 1]), np.ones([4, 1])), axis=1)
policy_random = np.array((np.random.permutation(2), np.random.permutation(2), np.random.permutation(2), np.random.permutation(2)))

# Initialize the environment
gamma = 0.9
env = StochasticCareerPathEnv()
is_Terminal = False
print("Initializing the environment for Stochastic Career Path Navigation...")
start_state = env.reset()
print()
steps = 0
total_reward = 0

# you may change policy here
chosen_policy_to_improve = policy_R
# policy = policy_1
# policy = policy_2
print("Initial Policy Details:")
print("State and Actions Values: \n", chosen_policy_to_improve)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(chosen_policy_to_improve[s])} ({env.action_names[np.argmax(chosen_policy_to_improve[s])]})")
print("State Transition Probabilities/Stochasticity:")
env.print_prob_dynamics()
print()

steps_to_take = 10
print("Calculating total rewards for this policy with the provided stochastic state transitions for", steps_to_take, "steps...")
print("State\t", "Action\t" , "New State\t" , "Reward\t" , "is_Terminal")
steps = 0
max_steps = 5

prev_state = start_state
while steps < steps_to_take:
    steps += 1

    action = np.random.choice(2,1,p=chosen_policy_to_improve[prev_state])[0]  #0 -> Research, 1 -> Development
    state, reward, is_Terminal = env.step(action)
    
    total_reward += reward

    print(" ",prev_state, "\t  ", action, "\t  ", state, "\t\t", reward, "\t", is_Terminal)
    prev_state = state
    
print("Total Number of steps:", steps_to_take)
print("Total Reward:", total_reward)

Initializing the environment for Stochastic Career Path Navigation...

Initial Policy Details:
State and Actions Values: 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 0 (Research)
State 1 (Industry): Best Action -> 0 (Research)
State 2 (Grad School): Best Action -> 0 (Research)
State 3 (Academia): Best Action -> 0 (Research)
State Transition Probabilities/Stochasticity:
State 0:
  Action 0:
    -> (Prob: 1.0, Next State: 2, Reward: 0.0, Non-terminal)
  Action 1:
    -> (Prob: 1.0, Next State: 1, Reward: 100.0, Non-terminal)
State 1:
  Action 0:
    -> (Prob: 0.9, Next State: 0, Reward: -10.0, Non-terminal)
    -> (Prob: 0.1, Next State: 1, Reward: 100, Non-terminal)
  Action 1:
    -> (Prob: 1.0, Next State: 1, Reward: 100.0, Non-terminal)
State 2:
  Action 0:
    -> (Prob: 0.9, Next State: 3, Reward: 10.0, Non-terminal)
    -> (Prob: 0.1, Next State: 2, Reward: 0, Non-terminal)
  Action 1:
    -> (Prob: 0.9, Next State: 1, Reward:

### B1.1 Find an optimal policy to navigate the given SCP environment using Policy Iteration (PI) 

In [22]:
# Policy Iteration for stochastic MDP
print("Comparison of PI and VI in Stochastic Transition MDP")
print("Finding optimal policy using Policy Iteration...")
pi_policy, pi_V, pi_iterations, pi_time = measure_convergence(env, chosen_policy_to_improve, "PI")
print(f"Policy Iteration: {pi_iterations} iterations, {pi_time:.6f} seconds")
print("Iterated Policy Details:")
print("State and Actions Values: \n", pi_policy)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(pi_policy[s])} ({env.action_names[np.argmax(pi_policy[s])]})")
print("State Values: ", pi_V)
print("Number of iterations: ", pi_iterations)

Comparison of PI and VI in Stochastic Transition MDP
Finding optimal policy using Policy Iteration...
Policy Iteration: 2 iterations, 0.017524 seconds
Iterated Policy Details:
State and Actions Values: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 1 (Development)
State 1 (Industry): Best Action -> 1 (Development)
State 2 (Grad School): Best Action -> 1 (Development)
State 3 (Academia): Best Action -> 1 (Development)
State Values:  [999.99999991 999.99999991 999.99999992 990.10989003]
Number of iterations:  2


### B1.2 Find an optimal policy to navigate the given SCP environment using Value Iteration (VI) 

In [23]:
# Value Iteration for stochastic MDP
# Measure convergence for Value Iteration
print("Finding optimal policy using Value Iteration...")
vi_policy, vi_V, vi_iterations, vi_time = measure_convergence(env, "", "VI")
print(f"Value Iteration: {vi_iterations} iterations, {vi_time:.6f} seconds")
print("Value Iterated Policy Details:")
print("State and Actions Values: \n", pi_policy)
# Render the policy in human readable form
print("State and Actions Names:")
for s in range(env.nS):
    print(f"State {s} ({env.state_names[s]}): Best Action -> {np.argmax(pi_policy[s])} ({env.action_names[np.argmax(pi_policy[s])]})")
print("State Values: ", pi_V)
print("Number of iterations: ", pi_iterations)

Finding optimal policy using Value Iteration...
Value Iteration: 220 iterations, 0.008166 seconds
Value Iterated Policy Details:
State and Actions Values: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
State and Actions Names:
State 0 (Unemployed): Best Action -> 1 (Development)
State 1 (Industry): Best Action -> 1 (Development)
State 2 (Grad School): Best Action -> 1 (Development)
State 3 (Academia): Best Action -> 1 (Development)
State Values:  [999.99999991 999.99999991 999.99999992 990.10989003]
Number of iterations:  2


### B1.3  Compare PI and VI in terms of convergence (average number of iteration, time required for each iteration). Is the policy obtained by both same for SCP environment?


In [24]:
# write your code for comparison here
policy_same = np.array_equal(vi_policy, pi_policy)
print(f"Policies obtained by VI and PI are the same: {policy_same}")

Policies obtained by VI and PI are the same: True


The observations for convergence - time taken, number of iterations and obtained polcies comparison are detailed in the output statements in the above cells.